In [1]:
import os
import config

import pandas as pd
import numpy as np
from scipy.stats import beta

from flask import Flask, jsonify, render_template

import mysql.connector

from datetime import datetime, date, time, timedelta
from time import sleep

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import requests

import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

# Fetch the service account key JSON file contents
cred = credentials.Certificate(config.firebase_file_path)
# Initialize the app with a service account, granting admin privileges
firebase_admin.initialize_app(cred, {
    'databaseURL': config.databaseURL
})


In [2]:
reviews_db = "reviews.json"
r = requests.get(config.databaseURL + reviews_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    review_df = pd.DataFrame.from_dict(data, orient='columns')

review_df.head()


,buyerID,date_reviewed
0,lmcq741,2019-07-24
1,cjoy123,2019-07-16
2,marissaspacey,2019-06-24
3,glyndasue5,2019-08-10
4,dick0390,2019-08-12


In [3]:
orders_db = "orders.json"
r = requests.get(config.databaseURL + orders_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    order_df = pd.DataFrame.from_dict(data, orient='columns')

order_df = order_df.astype({"orderID": int})

    
order_df

,buyerID,orderDate,orderID,orderTime,orderURL,record_date,shipByDate
0,zvq52mlo,2019-04-19,1441612324,12:04,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,
1,sshelburne12,2019-05-15,1461258703,6:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,
2,marissaspacey,2019-06-17,1465581068,10:00,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,
3,lmcq741,2019-06-21,1467289834,18:01,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,
4,admoorhead,2019-07-02,1471406642,22:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,
5,cjoy123,2019-07-04,1471997258,16:03,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-25
6,gauklerin09,2019-07-23,1479218984,21:31,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-30
7,dick0390,2019-07-09,1482734683,10:52,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-16
8,jessicaholder4,2019-07-20,1487089393,16:19,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,
9,,2019-07-26,1480128610,11:33,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-08-01


In [4]:
orderShipped_db = "order_shipped.json"
r = requests.get("https://jetchdashboard.firebaseio.com/" + orderShipped_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    orders_shipped_df = pd.DataFrame.from_dict(data, orient="columns")

orders_shipped_df = orders_shipped_df.astype({"orderID": int})

orders_shipped_df

,dateShipped,orderID
0,08/05/19,1482480540
1,07/30/19,1481079872
2,07/30/19,1481051598
3,07/27/19,1480128610
4,07/29/19,1479218984
5,07/21/19,1487089393
6,07/12/19,1482734683
7,07/12/19,1471997258
8,07/05/19,1471406642
9,07/05/19,1467289834


In [5]:
order_2 = pd.merge(order_df, orders_shipped_df, on="orderID", how="inner")
order_2

,buyerID,orderDate,orderID,orderTime,orderURL,record_date,shipByDate,dateShipped
0,zvq52mlo,2019-04-19,1441612324,12:04,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,04/28/19
1,sshelburne12,2019-05-15,1461258703,6:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,05/28/19
2,marissaspacey,2019-06-17,1465581068,10:00,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,06/20/19
3,lmcq741,2019-06-21,1467289834,18:01,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/05/19
4,admoorhead,2019-07-02,1471406642,22:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/05/19
5,cjoy123,2019-07-04,1471997258,16:03,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-25,07/12/19
6,gauklerin09,2019-07-23,1479218984,21:31,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-30,07/29/19
7,dick0390,2019-07-09,1482734683,10:52,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-16,07/12/19
8,jessicaholder4,2019-07-20,1487089393,16:19,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/21/19
9,,2019-07-26,1480128610,11:33,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-08-01,07/27/19


In [6]:
reviews2_db = "reviews2.json"
r = requests.get(config.databaseURL + reviews2_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    reviews2_df = pd.DataFrame.from_dict(data, orient='columns')

reviews2_df = reviews2_df.astype({"orderID": int})

reviews2_df.head()


,Name,email,orderID
0,Cierra Joy,cierrajoy582@yahoo.com,1471997258
1,Loren Day,lmcq74@yahoo.com,1467289834
2,Marissa Spacey,marissasdfghjkl@aim.com,1465581068
3,GLYNDA WHITE,glynda.sue.5@gmail.com,1481051598
4,Kadi Davis,dick0390@umn.edu,1482734683


In [7]:
df2=pd.merge(reviews2_df,order_2,on='orderID',how='outer')
df2

,Name,email,orderID,buyerID,orderDate,orderTime,orderURL,record_date,shipByDate,dateShipped
0,Cierra Joy,cierrajoy582@yahoo.com,1471997258,cjoy123,2019-07-04,16:03,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-25,07/12/19
1,Loren Day,lmcq74@yahoo.com,1467289834,lmcq741,2019-06-21,18:01,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/05/19
2,Marissa Spacey,marissasdfghjkl@aim.com,1465581068,marissaspacey,2019-06-17,10:00,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,06/20/19
3,GLYNDA WHITE,glynda.sue.5@gmail.com,1481051598,glyndasue5,2019-07-28,nan,nan,NaN,nan,07/30/19
4,Kadi Davis,dick0390@umn.edu,1482734683,dick0390,2019-07-09,10:52,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-16,07/12/19
5,NaN,NaN,1441612324,zvq52mlo,2019-04-19,12:04,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,04/28/19
6,NaN,NaN,1461258703,sshelburne12,2019-05-15,6:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,05/28/19
7,NaN,NaN,1471406642,admoorhead,2019-07-02,22:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/05/19
8,NaN,NaN,1479218984,gauklerin09,2019-07-23,21:31,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-30,07/29/19
9,NaN,NaN,1487089393,jessicaholder4,2019-07-20,16:19,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/21/19


In [8]:
df3=pd.merge(review_df,df2,on='buyerID',how='outer')
df3

,buyerID,date_reviewed,Name,email,orderID,orderDate,orderTime,orderURL,record_date,shipByDate,dateShipped
0,lmcq741,2019-07-24,Loren Day,lmcq74@yahoo.com,1467289834,2019-06-21,18:01,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/05/19
1,cjoy123,2019-07-16,Cierra Joy,cierrajoy582@yahoo.com,1471997258,2019-07-04,16:03,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-25,07/12/19
2,cjoy123,2019-07-16,NaN,NaN,1486172004,2019-08-10,14:33,https://www.etsy.com/your/orders/sold?ref=sell...,NaN,2019-08-30,08/19/19
3,marissaspacey,2019-06-24,Marissa Spacey,marissasdfghjkl@aim.com,1465581068,2019-06-17,10:00,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,06/20/19
4,glyndasue5,2019-08-10,GLYNDA WHITE,glynda.sue.5@gmail.com,1481051598,2019-07-28,nan,nan,NaN,nan,07/30/19
5,glyndasue5,2019-08-11,GLYNDA WHITE,glynda.sue.5@gmail.com,1481051598,2019-07-28,nan,nan,NaN,nan,07/30/19
6,dick0390,2019-08-12,Kadi Davis,dick0390@umn.edu,1482734683,2019-07-09,10:52,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,2019-07-16,07/12/19
7,zvq52mlo,NaN,NaN,NaN,1441612324,2019-04-19,12:04,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,04/28/19
8,sshelburne12,NaN,NaN,NaN,1461258703,2019-05-15,6:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,05/28/19
9,admoorhead,NaN,NaN,NaN,1471406642,2019-07-02,22:36,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-24,,07/05/19


In [9]:
df3=df3[['buyerID','date_reviewed','orderID','orderURL','orderDate','dateShipped']]
df3

,buyerID,date_reviewed,orderID,orderURL,orderDate,dateShipped
0,lmcq741,2019-07-24,1467289834,https://www.etsy.com/your/orders/sold?ref=sell...,2019-06-21,07/05/19
1,cjoy123,2019-07-16,1471997258,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-04,07/12/19
2,cjoy123,2019-07-16,1486172004,https://www.etsy.com/your/orders/sold?ref=sell...,2019-08-10,08/19/19
3,marissaspacey,2019-06-24,1465581068,https://www.etsy.com/your/orders/sold?ref=sell...,2019-06-17,06/20/19
4,glyndasue5,2019-08-10,1481051598,nan,2019-07-28,07/30/19
5,glyndasue5,2019-08-11,1481051598,nan,2019-07-28,07/30/19
6,dick0390,2019-08-12,1482734683,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-09,07/12/19
7,zvq52mlo,NaN,1441612324,https://www.etsy.com/your/orders/sold?ref=sell...,2019-04-19,04/28/19
8,sshelburne12,NaN,1461258703,https://www.etsy.com/your/orders/sold?ref=sell...,2019-05-15,05/28/19
9,admoorhead,NaN,1471406642,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-02,07/05/19


In [ ]:
df3['date_reviewed'].fillna(0, inplace=True)

In [21]:
df3 = df3.loc[df3["dateShipped"]!="nan",:]


In [22]:
df3['status']=''
df3.loc[df3['date_reviewed'] == 0, ['status']] = 'pending'
df3

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,buyerID,date_reviewed,orderID,orderURL,orderDate,dateShipped,status
0,lmcq741,2019-07-24,1467289834,https://www.etsy.com/your/orders/sold?ref=sell...,2019-06-21,07/05/19,
1,cjoy123,2019-07-16,1471997258,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-04,07/12/19,
2,cjoy123,2019-07-16,1486172004,https://www.etsy.com/your/orders/sold?ref=sell...,2019-08-10,08/19/19,
3,marissaspacey,2019-06-24,1465581068,https://www.etsy.com/your/orders/sold?ref=sell...,2019-06-17,06/20/19,
4,glyndasue5,2019-08-10,1481051598,nan,2019-07-28,07/30/19,
5,glyndasue5,2019-08-11,1481051598,nan,2019-07-28,07/30/19,
6,dick0390,2019-08-12,1482734683,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-09,07/12/19,
7,zvq52mlo,0,1441612324,https://www.etsy.com/your/orders/sold?ref=sell...,2019-04-19,04/28/19,pending
8,sshelburne12,0,1461258703,https://www.etsy.com/your/orders/sold?ref=sell...,2019-05-15,05/28/19,pending
9,admoorhead,0,1471406642,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-02,07/05/19,pending


In [11]:
pending_reviews=df3.loc[df3['status']=='pending',:]
pending_reviews


,buyerID,date_reviewed,orderID,orderURL,orderDate,dateShipped,status
7,zvq52mlo,0,1441612324,https://www.etsy.com/your/orders/sold?ref=sell...,2019-04-19,04/28/19,pending
8,sshelburne12,0,1461258703,https://www.etsy.com/your/orders/sold?ref=sell...,2019-05-15,05/28/19,pending
9,admoorhead,0,1471406642,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-02,07/05/19,pending
10,gauklerin09,0,1479218984,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-23,07/29/19,pending
11,jessicaholder4,0,1487089393,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-20,07/21/19,pending
12,jessicaholder4,0,1491061832,https://www.etsy.com/your/orders/sold?ref=sell...,2019-08-22,nan,pending
13,,0,1480128610,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-26,07/27/19,pending
14,Fudge69,0,1480472568,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-27,08/16/19,pending
15,brittanyb0710,0,1484941268,https://www.etsy.com/your/orders/sold?ref=sell...,2019-08-07,08/12/19,pending
16,HeatherW290,0,1493211993,https://www.etsy.com/your/orders/sold?ref=sell...,2019-08-05,08/19/19,pending


In [12]:
buyers_db = "buyers.json"
r = requests.get(config.databaseURL + buyers_db)
r = r.json()

if r:
    data = [r[i] for i in r]
    buyers_df = pd.DataFrame.from_dict(data, orient='columns')

buyers_df.head()


,buyerID,email,firstName,lastName,orderID,record_date
0,brittanyb0710,brittanybaggett@att.net,Brittany,Avants,1484941268,2019-08-08
1,HeatherW290,heathersemail@comcast.net,Heather Lynn,Wood,1493211993,2019-08-08
2,ivesterh89,nan,Hannah,Perez,1482480540,2019-08-08
3,laaia22,nan,Laaia,Albanese,1481079872,2019-08-08
4,glyndasue5,nan,GLYNDA,WHITE,1481051598,2019-08-08


In [13]:

buyers_df['lastInitial'] = buyers_df['lastName'].astype(str).str[0]
buyers_df['firstThree'] = buyers_df['firstName'].astype(str).str[:3]

buyers_df['name']=buyers_df['firstThree']+" "+buyers_df['lastInitial']

buyers_df=buyers_df[['name','orderID']]
buyers_df = buyers_df.astype({"orderID": int})

buyers_df


,name,orderID
0,Bri A,1484941268
1,Hea W,1493211993
2,Han P,1482480540
3,Laa A,1481079872
4,GLY W,1481051598
5,Kri n,1480472568
6,Bra G,1480128610
7,Hay J,1479218984
8,Jes H,1487089393
9,Kad D,1482734683


In [14]:
pending_reviews_df=pd.merge(buyers_df,pending_reviews,on='orderID',how='inner')
pending_reviews_df


,name,orderID,buyerID,date_reviewed,orderURL,orderDate,dateShipped,status
0,Bri A,1484941268,brittanyb0710,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-08-07,08/12/19,pending
1,Hea W,1493211993,HeatherW290,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-08-05,08/19/19,pending
2,Han P,1482480540,ivesterh89,0,nan,2019-08-01,08/05/19,pending
3,Laa A,1481079872,laaia22,0,nan,2019-07-29,07/30/19,pending
4,Kri n,1480472568,Fudge69,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-27,08/16/19,pending
5,Bra G,1480128610,,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-26,07/27/19,pending
6,Hay J,1479218984,gauklerin09,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-23,07/29/19,pending
7,Jes H,1487089393,jessicaholder4,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-20,07/21/19,pending
8,Ala M,1471406642,admoorhead,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-07-02,07/05/19,pending
9,Sha S,1461258703,sshelburne12,0,https://www.etsy.com/your/orders/sold?ref=sell...,2019-05-15,05/28/19,pending


In [15]:
import os, time
time.strftime('%X %x %Z')

os.environ['TZ'] = 'Europe/London'
time.tzset()

time.strftime('%X %x %Z')

'18:12:40 08/22/19 BST'

In [17]:
from pytz import timezone

now = datetime.now()
now = timezone('US/Central').localize(now)
pending_reviews_df['now'] = now

for index, item in pending_reviews_df.iterrows():
    print(item.dateShipped)
    dateShipped = datetime.strptime(item.dateShipped, "%m/%d/%y")
    dateShipped = timezone('US/Central').localize(dateShipped)

    dateShipped =dateShipped + timedelta(hours=5)


    pending_reviews_df.loc[index,'dateShipped_local']=dateShipped

pending_reviews_df


08/12/19
08/19/19
08/05/19
07/30/19
08/16/19
07/27/19
07/29/19
07/21/19
07/05/19
05/28/19
04/28/19
08/19/19
08/19/19
nan


ValueError: time data 'nan' does not match format '%m/%d/%y'

In [ ]:
for index, item in pending_reviews_df.iterrows():
    days=(item.now-item.dateShipped_local).total_seconds()
    d = days//86400
    pending_reviews_df.loc[index,'daysSinceShipped']=d

pending_reviews_df

In [ ]:
reviews_under_100=pending_reviews_df.loc[pending_reviews_df['daysSinceShipped']<=100,:]
reviews_under_100=reviews_under_100.sort_values(by='daysSinceShipped', ascending=False)
reviews_under_100 = reviews_under_100.astype({"daysSinceShipped": int})

reviews_under_100

In [ ]:
reviews_under_100.dtypes

In [ ]:
outstanding_reviews=[]

for index, item in reviews_under_100.iterrows():
    reviewdict={}
    reviewdict['a_buyerName']=item[0]
    reviewdict['c_orderURL']=item[4]
    reviewdict['b_daysSinceShipped']= item[10]
    outstanding_reviews.append(reviewdict)

outstanding_reviews